In [2]:
!pip install transformers

     |████████████████████████████████| 2.5MB 30.6MB/s 
     |████████████████████████████████| 3.3MB 32.9MB/s 
     |████████████████████████████████| 901kB 42.0MB/s 


In [1]:
import torch

In [3]:
!pip install datasets

     |████████████████████████████████| 266kB 31.3MB/s 
     |████████████████████████████████| 245kB 34.6MB/s 
     |████████████████████████████████| 122kB 45.1MB/s 


In [4]:
!pip install scikit-learn

In [5]:
from transformers import AutoTokenizer
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from datasets import load_metric

In [7]:
dataset = load_dataset('csv', data_files=['twitter15_text.csv'])['train'].train_test_split(test_size=0.25)

Using custom data configuration default-4024d13d5c4c310e


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4024d13d5c4c310e/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


In [8]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def tokenize(batch):
  tokens = tokenizer(batch['text'], padding=True, truncation=True, max_length=128)
  tokens['labels'] = labels.str2int(batch['labels'])
  return tokens

In [10]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [11]:
training_set = tokenized_datasets['train']#.select(range(100))
test_set = tokenized_datasets['test']#.select(range(100))


In [17]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4, )

training_args = TrainingArguments("./test_trainer", evaluation_strategy="epoch", num_train_epochs=10.0)


metric = load_metric("accuracy")


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab

In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics
)

In [19]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running training *****
  Num examples = 1117
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1400


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.919608,0.643432
2,No log,0.624791,0.793566
3,No log,0.878862,0.804290
4,0.591600,1.037331,0.812332
5,0.591600,1.140359,0.806971
6,0.591600,1.113599,0.831099
7,0.591600,1.133789,0.839142
8,0.008200,1.149251,0.844504
9,0.008200,1.159016,0.844504
10,0.008200,1.162517,0.844504


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running Evaluation *****
  Num examples = 373
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running Evaluation *****
  Num examples = 373
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running Evaluation *****
  Num examples = 373
  Batch size = 8
Saving model checkpoint to ./test_trainer/checkpoint-500
Configuration saved in ./test_trainer/checkpoint-500/config.json
Model weights saved in ./test_trainer/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and h

TrainOutput(global_step=1400, training_loss=0.21431647725403308, metrics={'train_runtime': 1294.0372, 'train_samples_per_second': 8.632, 'train_steps_per_second': 1.082, 'total_flos': 3756905409699840.0, 'train_loss': 0.21431647725403308, 'epoch': 10.0})

In [20]:
metrics=trainer.evaluate()
print(metrics)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running Evaluation *****
  Num examples = 373
  Batch size = 8


{'eval_loss': 1.1625168323516846, 'eval_accuracy': 0.8445040214477212, 'eval_runtime': 13.4774, 'eval_samples_per_second': 27.676, 'eval_steps_per_second': 3.487, 'epoch': 10.0}
